# 1) Checks from NC files

In [2]:
import pandas as pd

test = pd.read_csv('/Users/paolo/Documents/TC/data/test_real_tom_target.csv')
train = pd.read_csv('/Users/paolo/Documents/TC/data/training_real_tom_target.csv')
val = pd.read_csv('/Users/paolo/Documents/TC/data/validation_real_tom_target.csv')

In [3]:
train.groupby(['latitude','longitude']).count() #11.323 days 
val.groupby(['latitude','longitude']).count() #1.826 days 
test.groupby(['latitude','longitude']).count() #1.431 days 
# 377 rows, 8 input variables 

Unnamed: 0  time    vo     r  u_200  u_850  v_200  v_850  \
latitude longitude                                                             
-30.0    20.0             1431  1431  1431  1431   1431   1431   1431   1431   
         22.5             1431  1431  1431  1431   1431   1431   1431   1431   
         25.0             1431  1431  1431  1431   1431   1431   1431   1431   
         27.5             1431  1431  1431  1431   1431   1431   1431   1431   
         30.0             1431  1431  1431  1431   1431   1431   1431   1431   
...                        ...   ...   ...   ...    ...    ...    ...    ...   
 0.0     80.0             1431  1431  1431  1431   1431   1431   1431   1431   
         82.5             1431  1431  1431  1431   1431   1431   1431   1431   
         85.0             1431  1431  1431  1431   1431   1431   1431   1431   
         87.5             1431  1431  1431  1431   1431   1431   1431   1431   
         90.0             1431  1431  1431  1431   1431   1431   1431   1431   

                     ttr   sst   lsm  Real_tom_lsm  
latitude longitude                                  
-30.0    20.0       1431  1431  1431          1431  
         22.5       1431  1431  1431          1431  
         25.0       1431  1431  1431          1431  
         27.5       1431  1431  1431          1431  
         30.0       1431  1431  1431          1431  
...                  ...   ...   ...           ...  
 0.0     80.0       1431  1431  1431          1431  
         82.5       1431  1431  1431          1431  
         85.0       1431  1431  1431          1431  
         87.5       1431  1431  1431          1431  
         90.0       1431  1431  1431          1431  

[377 rows x 12 columns]

In [4]:
def check_correct_lsm(dataset):
    print(dataset.head())

    lats = dataset.latitude.unique()
    lons = dataset.longitude.unique()
    
    print(len(lats), len(lons))
    
    check=0
    for lat in lats:
        for lon in lons:
            dummy_df = dataset.loc[(dataset.latitude==lat) & (dataset.longitude==lon)].sort_values('time',axis=0, ascending=True)
            #print(len(dummy_df))
            s = sum(abs(dummy_df.loc[:,'Real_tom_lsm'].iloc[:-1].values-dummy_df.loc[:,'lsm'].iloc[1:].values))
            if s>0: print('error') 
            #print(dummy_df.loc[1:,'lsm'].values)
#train.loc[(train.latitude==-17.5) & (train.longitude==85) & (train.time<='1980-12-31') & (train.lsm+train.Real_tom_lsm>=1)]


In [5]:
check_correct_lsm(train) 
check_correct_lsm(val) 
check_correct_lsm(test) 
# ok

   Unnamed: 0        time  latitude  longitude        vo          r     u_200  \
0           0  1980-01-01       0.0       20.0  0.000007  80.761185  1.909660   
1           1  1980-01-01       0.0       22.5  0.000004  80.703650  1.165733   
2           2  1980-01-01       0.0       25.0  0.000007  78.231514 -1.311676   
3           3  1980-01-01       0.0       27.5  0.000010  79.631010 -3.777573   
4           4  1980-01-01       0.0       30.0  0.000010  71.573875 -5.734505   

      u_850     v_200     v_850        ttr  sst  lsm  Real_tom_lsm  
0 -3.323872  1.687164 -1.823624 -247.54074  0.0  0.0           0.0  
1 -2.844494  1.060593 -1.991425 -240.00592  0.0  0.0           0.0  
2 -2.125244  3.280617 -1.931789 -223.76889  0.0  0.0           0.0  
3 -1.122395  5.743889 -1.243538 -235.55556  0.0  0.0           0.0  
4 -1.362953  6.514030 -0.954163 -254.03260  0.0  0.0           0.0  
13 29
   Unnamed: 0        time  latitude  longitude        vo          r     u_200  \
0           

In [6]:
# check features with file nc

import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt


def open_file(path='', eng=None, backed=False, key='', multiple=False, multiple_value=0.0, lat=0, lon=0):
    if backed==True:
        ds = xr.open_dataset(path, engine=eng, backend_kwargs={'filter_by_keys': {'shortName': key}})
    else: ds = xr.open_dataset(path, engine=eng)
        
    gpi_bidim = ds.to_dataframe()

    # print and checks
    #print(gpi_bidim)
    #print(gpi_bidim.step.unique())
    #print(gpi_bidim.number.unique())
    #if 'isobaricInhPa' in gpi_bidim.columns: print(gpi_bidim.isobaricInhPa.unique())
    
    # reload and see number of data
    if backed==True:
        ds = xr.open_dataset(path, engine=eng, backend_kwargs={'filter_by_keys': {'shortName': key}})
    else: ds = xr.open_dataset(path, engine=eng)
    gpi_bidim = ds.to_dataframe()
    gpi_bidim = gpi_bidim.reset_index()
    if multiple is False: 
        gpi_bidim = gpi_bidim.loc[:,['time','latitude','longitude',key]]
    else: 
        gpi_bidim = gpi_bidim.loc[gpi_bidim.isobaricInhPa==multiple_value,['time','latitude','longitude',key]]

    res = gpi_bidim.groupby(['time']).count()
    #print(res)
    #print(res.loc[res[key]!=10512])
    
    # plot di un punto ogni giorno
    #plt.plot(gpi_bidim.loc[(gpi_bidim.latitude==lat) & (gpi_bidim.longitude==lon),key].reset_index(drop=True))
    
    # check 
    return gpi_bidim.loc[(gpi_bidim.latitude==lat) & (gpi_bidim.longitude==lon),:]

In [7]:
test = pd.read_csv('/Users/paolo/Documents/TC/data/test_real_tom_target.csv')
train = pd.read_csv('/Users/paolo/Documents/TC/data/training_real_tom_target.csv')
val = pd.read_csv('/Users/paolo/Documents/TC/data/validation_real_tom_target.csv')

In [8]:
lats = test.latitude.unique()
lons = test.longitude.unique()

for lat in lats:
    for lon in lons:
        dummy_df_true = open_file(path="/Users/paolo/Documents/TC_old/data/data_new_samples/20190101_20191231.grb", eng='cfgrib', backed=True, key='vo', lat=lat, lon=lon)
        dummy_df_true = dummy_df_true.loc[dummy_df_true.time<='2019-12-01']
        dummy_df = test.loc[(test.latitude==lat) & (test.longitude==lon) & (test.time<='2019-12-31') & (test.time>='2019-01-01')].sort_values('time',axis=0, ascending=True)
        s = (sum(abs(dummy_df['vo'].values-dummy_df_true['vo'].values)))
        if s > 10**(-9): print(s)
        

In [9]:
for lat in lats:
    for lon in lons:
        dummy_df_true = open_file(path="/Users/paolo/Documents/TC_old/data/data_new_samples/20190101_20191231.grb", eng='cfgrib', backed=True, key='r', lat=lat, lon=lon)
        dummy_df_true = dummy_df_true.loc[dummy_df_true.time<='2019-12-01']
        dummy_df = test.loc[(test.latitude==lat) & (test.longitude==lon) & (test.time<='2019-12-31') & (test.time>='2019-01-01')].sort_values('time',axis=0, ascending=True)
        s = (sum(abs(dummy_df['r'].values-dummy_df_true['r'].values)))
        if s > 10**(-3): print(s)

In [10]:
for lat in lats:
    for lon in lons:
        dummy_df_true = open_file(path="/Users/paolo/Documents/TC_old/data/data_new_samples/20190101_20191231.grb", eng='cfgrib', backed=True, key='u', multiple=True, multiple_value=850.0, lat=lat, lon=lon)
        dummy_df_true = dummy_df_true.loc[dummy_df_true.time<='2019-12-01']
        dummy_df = test.loc[(test.latitude==lat) & (test.longitude==lon) & (test.time<='2019-12-31') & (test.time>='2019-01-01')].sort_values('time',axis=0, ascending=True)
        s = (sum(abs(dummy_df['u_850'].values-dummy_df_true['u'].values)))
        if s > 10**(-4): print(s)

In [11]:
for lat in lats:
    for lon in lons:
        dummy_df_true = open_file(path="/Users/paolo/Documents/TC_old/data/data_new_samples/20190101_20191231.grb", eng='cfgrib', backed=True, key='u', multiple=True, multiple_value=200.0, lat=lat, lon=lon)
        dummy_df_true = dummy_df_true.loc[dummy_df_true.time<='2019-12-01']
        dummy_df = test.loc[(test.latitude==lat) & (test.longitude==lon) & (test.time<='2019-12-31') & (test.time>='2019-01-01')].sort_values('time',axis=0, ascending=True)
        s = (sum(abs(dummy_df['u_200'].values-dummy_df_true['u'].values)))
        if s > 10**(-3): print(s)

In [13]:
for lat in lats:
    for lon in lons:
        dummy_df_true = open_file(path="/Users/paolo/Documents/TC_old/data/data_new_samples/20190101_20191231.grb", eng='cfgrib', backed=True, key='v', multiple=True, multiple_value=850.0, lat=lat, lon=lon)
        dummy_df_true = dummy_df_true.loc[dummy_df_true.time<='2019-12-01']
        dummy_df = test.loc[(test.latitude==lat) & (test.longitude==lon) & (test.time<='2019-12-31') & (test.time>='2019-01-01')].sort_values('time',axis=0, ascending=True)
        s = (sum(abs(dummy_df['v_850'].values-dummy_df_true['v'].values)))
        if s > 10**(-3): print(s)

In [14]:
for lat in lats:
    for lon in lons:
        dummy_df_true = open_file(path="/Users/paolo/Documents/TC_old/data/data_new_samples/20190101_20191231.grb", eng='cfgrib', backed=True, key='v', multiple=True, multiple_value=200.0, lat=lat, lon=lon)
        dummy_df_true = dummy_df_true.loc[dummy_df_true.time<='2019-12-01']
        dummy_df = test.loc[(test.latitude==lat) & (test.longitude==lon) & (test.time<='2019-12-31') & (test.time>='2019-01-01')].sort_values('time',axis=0, ascending=True)
        s = (sum(abs(dummy_df['v_200'].values-dummy_df_true['v'].values)))
        if s > 10**(-3): print(s)

In [15]:
for lat in lats:
    for lon in lons:
        dummy_df_true = open_file(path="/Users/paolo/Documents/TC_old/data/data_new_samples/2d_sst_trop_20190101_20191231.grb", eng='cfgrib', backed=False, key='sst', multiple=False, lat=lat, lon=lon)
        dummy_df_true = dummy_df_true.loc[dummy_df_true.time<='2019-12-01']
        dummy_df = test.loc[(test.latitude==lat) & (test.longitude==lon) & (test.time<='2019-12-31') & (test.time>='2019-01-01')].sort_values('time',axis=0, ascending=True)
        s = (sum(abs(dummy_df['sst'].values-dummy_df_true['sst'].values)))
        if s > 10**(-2): print(s)

In [16]:
for lat in lats:
    for lon in lons:
        dummy_df_true = open_file(path="/Users/paolo/Documents/TC_old/data/data_new_samples/an_20190101_20191231_25.grb", eng='cfgrib', backed=False, key='ttr', multiple=False, lat=lat, lon=lon)
        dummy_df_true = dummy_df_true.loc[dummy_df_true.time<='2019-12-01']
        dummy_df = test.loc[(test.latitude==lat) & (test.longitude==lon) & (test.time<='2019-12-31') & (test.time>='2019-01-01')].sort_values('time',axis=0, ascending=True)
        s = (sum(abs(dummy_df['ttr'].values-dummy_df_true['ttr'].values)))
        if s > 10**(-2): print(s)